In [55]:
import time
import numpy as np
import pandas as pd
from tqdm import tqdm
from bs4 import BeautifulSoup
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from selenium import webdriver
from selenium.webdriver.chrome.service import Service

browser  = webdriver.Chrome()

### 1. Multi

In [57]:
data = []

for page in tqdm(range(0,1)):

    
    # link = 'https://www.naukri.com/jobs-in-india-' + str(page) 
    link = 'https://www.naukri.com/jobs-in-india?functionAreaIdGid=5&experience=0' + str(page)
    browser.get(link)
    time.sleep(10)
    
    soup = BeautifulSoup(browser.page_source, 'html.parser')
    
    for job in soup.find_all('div', class_='srp-jobtuple-wrapper'): 

        try:
            job_id = job.get('data-job-id').strip()
        except:
            job_id = np.nan

        try:
            job_role = job.find('a', class_='title').text.strip()
        except:
            job_role = np.nan

        try:
            job_link = job.find('a', class_='title').get('href')
        except:
            job_link = np.nan

        try:
            reviews = job.find('a', class_='review ver-line').text.strip()
        except:
            reviews = np.nan

        try:
            rating = job.find('a', class_='rating').text.strip()
        except:
            rating = np.nan

        try:
            company = job.find('a', class_='comp-name').text.strip()
        except:
            company = np.nan

        try:
            company_link = job.find('a', class_='comp-name').get('href')
        except:
            company_link = np.nan

        try:
            experience = job.find('span', class_='expwdth').text.strip()
        except:
            experience = np.nan

        try:
            salary = job.find('span', class_='sal-wrap').text.strip()
        except:
            salary = np.nan

        try:
            location = job.find('span', class_='locWdth').text.strip()
        except:
            location = np.nan

        try:
            responsibilities = ','.join([i.text.strip() for i in job.find('ul', class_='tags-gt').find_all('li')])
        except:
            responsibilities = np.nan

        try:
            posted_on = job.find('span', class_='job-post-day').text.strip()
        except:
            posted_on = np.nan

        try:
            job_description = job.find('span', class_='job-desc').text.strip()
        except:
            job_description = np.nan

        try:
            tags = ', '.join([tag.text for tag in job.find_all('li', class_='dot-gt')])
        except:
            tags = np.nan

        try:
            company_logo_url = job.find('img', class_='logoImage')['src']
        except:
            company_logo_url = np.nan

        # data.append([job_id, job_role, company, experience, salary, location, rating, reviews, 
        #              responsibilities, posted_on, job_link, company_link, job_description, tags, company_logo_url])
        data.append([job_role, company, 
                     responsibilities, job_link, job_description])
        
# df = pd.DataFrame(data, columns=['job_id', 'job_role', 'company', 'experience', 'salary', 'location', 
#                                  'rating', 'reviews', 'responsibilities', 'posted_on', 'job_link', 
#                                  'company_link', 'job_description', 'tags', 'company_logo_url'])

df = pd.DataFrame(data, columns=['job_role', 'company', 'responsibilities', 'job_link', 'job_description'])

df

df.to_csv('jobs.csv', index = False)

scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']

creds = ServiceAccountCredentials.from_json_keyfile_name('google_credentials.json', scope)

client = gspread.authorize(creds)

sheet = client.open('Google-Sheets-Integration').sheet1

df = pd.read_csv('jobs.csv')

df = df.fillna('')

data_to_upload = [df.columns.values.tolist()] + df.values.tolist()

sheet.clear()

sheet.update(data_to_upload)


100%|██████████| 1/1 [00:10<00:00, 10.38s/it]


{'spreadsheetId': '1xHgHBj277Bg9g0RRMAAIhIPL1ud6iEr8QLQV4jWXxGM',
 'updatedRange': 'Sheet1!A1:E21',
 'updatedRows': 21,
 'updatedColumns': 5,
 'updatedCells': 105}